In [11]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [12]:
versi_model=6
df = pd.read_csv('../Dataset/dataset_2.csv')
df


,Ulasan,Label
0,not bad,Neutral
1,beli bintang bicara,Neutral
2,alas warna kuning luntur kaos kaki putih,Neutral
3,harga segini worth sih kesalahan cuman salah m...,Neutral
4,ukurannya sih kayanya pake sepatu adidas new b...,Neutral
...,...,...
1149,sepatu belah kiri tekuk utk tumit,Tidak Puas
1150,layan ratih listya buruk,Tidak Puas
1151,jelek barangnya,Tidak Puas
1152,kirim lamaa bangetttt,Tidak Puas


In [13]:
# Ubah label -1 menjadi 2
df['Label'] = df['Label'].map({"Puas": 2, "Neutral": 1, "Tidak Puas": 0})

# Validasi Label
assert df['Label'].isin([0, 1, 2]).all(), "Ada Label yang tidak valid"
print(df['Label'].unique())  # Output harus [0, 1, 2]


[1 2 0]


In [14]:
# Tokenisasi Data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128)

df['tokenized'] = df['Ulasan'].apply(lambda x: tokenize_function(x))


In [15]:
# Split Dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(df['Ulasan'], df['Label'], test_size=0.2, random_state=42)


In [16]:
# Buat Dataset PyTorch
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=128)

train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
val_dataset = SentimentDataset(val_encodings, val_labels.tolist())


In [17]:
# Ubah BERT dan Trainer ke mode GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [18]:

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.to('cuda')
training_args = TrainingArguments(
    output_dir='../results/v_' + str(versi_model),
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=400,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=20,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=200,
    save_total_limit=2,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  3%|▎         | 20/580 [00:08<03:32,  2.64it/s]

{'loss': 1.1225, 'grad_norm': 5.3195576667785645, 'learning_rate': 2.5e-06, 'epoch': 0.34}


  7%|▋         | 40/580 [00:16<03:25,  2.62it/s]

{'loss': 1.0944, 'grad_norm': 5.547304630279541, 'learning_rate': 5e-06, 'epoch': 0.69}


 10%|█         | 60/580 [00:23<03:11,  2.71it/s]

...

 97%|█████████▋| 560/580 [03:45<00:07,  2.60it/s]

{'loss': 0.0479, 'grad_norm': 0.08613096922636032, 'learning_rate': 5.555555555555556e-06, 'epoch': 9.66}


100%|██████████| 580/580 [03:53<00:00,  2.48it/s]

{'loss': 0.0402, 'grad_norm': 0.04435143247246742, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 233.5398, 'train_samples_per_second': 39.522, 'train_steps_per_second': 2.484, 'train_loss': 0.4413453757762909, 'epoch': 10.0}


TrainOutput(global_step=580, training_loss=0.4413453757762909, metrics={'train_runtime': 233.5398, 'train_samples_per_second': 39.522, 'train_steps_per_second': 2.484, 'train_loss': 0.4413453757762909, 'epoch': 10.0})

In [19]:
# Evaluasi model setelah pelatihan
eval_result = trainer.evaluate()
eval_result

100%|██████████| 8/8 [00:00<00:00,  8.49it/s]


{'eval_loss': 0.9896822571754456,
 'eval_accuracy': 0.8181818181818182,
 'eval_f1': 0.8181887078438802,
 'eval_precision': 0.8270822230797973,
 'eval_recall': 0.8162367610940088,
 'eval_runtime': 1.0868,
 'eval_samples_per_second': 212.55,
 'eval_steps_per_second': 7.361,
 'epoch': 10.0}

In [20]:

# Save model
print("Model berhasil disimpan di direktori ../results/" + "v_" + str(versi_model))
model.save_pretrained("../results/" + "v_" + str(versi_model))
tokenizer.save_pretrained("../results/" + "v_" + str(versi_model))

Model berhasil disimpan di direktori ../results/v_6


('../results/v_6\\tokenizer_config.json',
 '../results/v_6\\special_tokens_map.json',
 '../results/v_6\\vocab.txt',
 '../results/v_6\\added_tokens.json')